# Graph measures

This script is used to calculate relevant graph metrics on full connectivity matrices.

### Graph metrics

Graph measures related to modularity are calculated. First, community structure is discovered using `community_louvain` algorithm for undirected, weighted networks with positive connections. Canonical **modularity** quality function is used throughout the search. Module size is controlled by the `louvain_gamma` resolution parameter (default 1). Search is conducted `louvain_reps` times and the division with highest score is stored as a representative for a network. Then node-level measures of centrality – **within-module degree z-score** and **participation coeffiecient** (measuring diversity of intermodular connections) are calculated for each network.

In [1]:
import json
import os
from os.path import join
from itertools import product
from pathlib import Path

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

from bct.algorithms.modularity import community_louvain
from bct.algorithms.clustering import clustering_coef_wu
from bct.algorithms.reference import randmio_und_signed
from bct.algorithms.centrality import module_degree_zscore, participation_coef
from bct.algorithms.distance import efficiency_wei
from dn_utils.plotting import plot_matrix
from scipy.stats import ttest_1samp
from statsmodels.stats.multitest import fdrcorrection

%matplotlib inline

/home/kmb/Desktop/Neuroscience/Projects/BONNA_decide_net/code/dn_utils/glm_utils.py:13: FutureWarning: 

 | Starting with Nilearn 0.7.0, all Nistats functionality has been incorporated into Nilearn's stats & reporting modules.
 | Nistats package will no longer be updated or maintained.

  from nistats import design_matrix


In [2]:
path_root = os.environ.get('DECIDENET_PATH')

path_derivatives = join(path_root, 'data/main_fmri_study/derivatives')
path_sourcedata = join(path_root, 'data/main_fmri_study/sourcedata') 

path_beh = join(path_sourcedata, 'behavioral')
path_bsc = join(path_derivatives, 'bsc')
path_nistats = join(path_derivatives, 'nistats')
path_parcellations = join(path_derivatives, 'parcellations')

path_corrmats = join(path_bsc, 'corrmats')

## Settings

In [3]:
atlas = 'combined_roi'

# Modularity
louvain_B = 'negative_asym'
louvain_reps = 100
gamma_range = np.arange(0.5, 3.5, 0.25)

# Create output paths
path_out = join(path_corrmats, atlas, f"unthr")
Path(path_out).mkdir(exist_ok=True, parents=True)

### Load data

In [4]:
# Load correlation matrices and metadata
corrmats_aggregated = np.load(join(path_corrmats, atlas, 
                                   'corrmats_aggregated.npy'))
with open(join(path_corrmats, atlas, 'corrmats_aggregated.json'), 'r') as f:
    meta = json.loads(f.read())

# Load subject exclusion
df_exclusion = pd.read_csv(join(path_nistats, 'exclusion/exclusion.csv'), 
                           index_col=0)
ok_index = df_exclusion['ok_all']    
    
# Load ROI information
df_roi = pd.read_csv(join(path_corrmats, atlas, 'roi_table_filtered.csv'))
    
n_subjects = len(meta['dim1'])
n_conditions = len(meta['dim2'])
n_perr_sign = len(meta['dim3'])
n_rois = len(df_roi)

### Calculate measures

In [5]:
for gamma in gamma_range:
    
    print(f"Calculating graph measures for 𝛾 = {gamma}")
    
    gamma_str = str(float(gamma)).replace('.', '_')
    path_out_gamma = join(path_out, f"gamma_{gamma_str}")
    Path(path_out_gamma).mkdir(exist_ok=True)

    m_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign, n_rois))
    q_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign))
    z_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign, n_rois))
    p_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign, n_rois))

    iters = product(range(n_subjects), range(n_conditions), range(n_perr_sign))
    for sub_idx, con_idx, perr_sign_idx in tqdm(list(iters)):

        corrmat = corrmats_aggregated[sub_idx, con_idx, perr_sign_idx]
        corrmat[np.diag_indices_from(corrmat)] = 0

        best_q = 0 
        for _ in range(louvain_reps):
            m, q = community_louvain(corrmat, gamma=gamma, B=louvain_B)
            if q > best_q:
                best_m = m
                best_q = q

        # Within-module degree z-score
        z_aggregated[sub_idx, con_idx, perr_sign_idx] = module_degree_zscore(
            W=corrmat, ci=best_m, flag=0)

        # Participation coefficient
        p_aggregated[sub_idx, con_idx, perr_sign_idx] = participation_coef(
            W=corrmat, ci=best_m, degree="undirected")

        # Store best values
        m_aggregated[sub_idx, con_idx, perr_sign_idx] = best_m
        q_aggregated[sub_idx, con_idx, perr_sign_idx] = best_q


    np.save(join(path_out_gamma, f"m_aggregated.npy"), m_aggregated)
    np.save(join(path_out_gamma, f"q_aggregated.npy"), q_aggregated)
    np.save(join(path_out_gamma, f"z_aggregated.npy"), z_aggregated)
    np.save(join(path_out_gamma, f"p_aggregated.npy"), p_aggregated)

Calculating graph measures for 𝛾 = 0.5


  0%|          | 0/128 [00:00<?, ?it/s]

/home/kmb/miniconda3/envs/decidenet/lib/python3.8/site-packages/bct/algorithms/centrality.py:661: RuntimeWarning: invalid value encountered in true_divide
  Z[np.where(ci == i)] = (Koi - np.mean(Koi)) / np.std(Koi)


Calculating graph measures for 𝛾 = 0.75


  0%|          | 0/128 [00:00<?, ?it/s]

...

  0%|          | 0/128 [00:00<?, ?it/s]

Calculating graph measures for 𝛾 = 3.0


  0%|          | 0/128 [00:00<?, ?it/s]

Calculating graph measures for 𝛾 = 3.25


  0%|          | 0/128 [00:00<?, ?it/s]